# Examples of using analysis functionalities

Using discovery_utils analyses functionalities for investments data

Here, we'll find companies using their categories, but you can also use search results from the process shown in cybersec_search.ipynb

In [ ]:
from discovery_utils.utils import (
    viz_landscape
)

In [ ]:
import pandas as pd
from src import PROJECT_DIR

In [ ]:
from discovery_utils.getters import crunchbase
CB = crunchbase.CrunchbaseGetter(vector_db_path=PROJECT_DIR / "tmp/vector_db")

In [ ]:
lists_of_categories = {
    "biological": ["Agtech", "Agriculture"],
    "health_diabetes": ["Diabetes"],
    "health_nutrition": ["Dietary Supplements", "Nutrition"],
    "economic_retail": ["Organic Food", "Grocery", "Snack Food", "Farmers Market"],
    "economic_ooh": ["Food Delivery", "Restaurants"],
    "economic_food_proc": ["Food Processing"],
    "food_beverage": ["Food and Beverage"],
    "social": ["Recipes"]
}

In [ ]:
save_name = "social"
list_of_categories = lists_of_categories[save_name]
print(list_of_categories)
selected_df = CB.get_companies_in_categories(list_of_categories, category_type="narrow")

matching_ids = set(list(selected_df.id.to_list()))

In [ ]:
only_recent_companies = False
if only_recent_companies:
    ids_recent = CB.organisations_enriched.query("last_funding_on > '2019'").id.to_list()
    ids_new = CB.organisations_enriched.query("founded_on > '2019'").id.to_list()
    recent_or_new = set(ids_recent + ids_new)
    matching_ids = matching_ids.intersection(recent_or_new)

In [ ]:
len(matching_ids)

In [ ]:
# write an sql query to achieve id in test_ids
id_condition = "id in ('{}')".format("', '".join(list(matching_ids)))
vectors_df = CB.VectorDB.vector_db.search().where(id_condition).limit(30000).to_pandas()
len(vectors_df)

In [ ]:
fig, cb_viz_df = viz_landscape.generate_crunchbase_landscape(vectors_df, CB)

In [ ]:
output_path = PROJECT_DIR / f'data/2025_01_MS_ahl/_landscape_{save_name}.html'
# cb_viz_df.to_csv(PROJECT_DIR / f"data/2025_01_MS_ahl/_table_{save_name}.csv", index=False)
fig.save(str(output_path))